# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 27 2022 9:12AM,253803,10,MSP217025,"Methapharm, Inc.",Released
1,Dec 27 2022 9:12AM,253803,10,MSP217029,"Methapharm, Inc.",Released
2,Dec 27 2022 9:11AM,253798,10,0086268699,ISDIN Corporation,Released
3,Dec 27 2022 9:11AM,253798,10,0086268700,ISDIN Corporation,Released
4,Dec 27 2022 9:11AM,253798,10,0086268701,ISDIN Corporation,Released
5,Dec 27 2022 9:11AM,253798,10,0086268704,ISDIN Corporation,Released
6,Dec 27 2022 9:11AM,253798,10,0086268703,ISDIN Corporation,Released
7,Dec 27 2022 9:11AM,253798,10,0086268713,ISDIN Corporation,Released
8,Dec 27 2022 9:11AM,253798,10,0086268712,ISDIN Corporation,Released
9,Dec 27 2022 9:11AM,253798,10,0086268716,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,253797,Released,1
13,253798,Released,40
14,253799,Released,1
15,253802,Released,1
16,253803,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253797,NaN,1.0
253798,NaN,40.0
253799,NaN,1.0
253802,NaN,1.0
253803,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253781,0.0,2.0
253783,0.0,1.0
253784,6.0,23.0
253785,0.0,1.0
253786,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253781,0,2
253783,0,1
253784,6,23
253785,0,1
253786,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253781,0,2
1,253783,0,1
2,253784,6,23
3,253785,0,1
4,253786,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253781,,2
1,253783,,1
2,253784,6,23
3,253785,,1
4,253786,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc."
2,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation
42,Dec 27 2022 9:10AM,253802,10,"Methapharm, Inc."
43,Dec 27 2022 9:03AM,253799,10,Bio-PRF
44,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation
105,Dec 27 2022 9:02AM,253797,12,Hush Hush
106,Dec 27 2022 9:01AM,253796,10,Bio-PRF
107,Dec 27 2022 8:55AM,253794,10,ISDIN Corporation
125,Dec 27 2022 8:49AM,253790,10,ISDIN Corporation
130,Dec 27 2022 8:38AM,253789,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc.",,2
1,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,,40
2,Dec 27 2022 9:10AM,253802,10,"Methapharm, Inc.",,1
3,Dec 27 2022 9:03AM,253799,10,Bio-PRF,,1
4,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation,,61
5,Dec 27 2022 9:02AM,253797,12,Hush Hush,,1
6,Dec 27 2022 9:01AM,253796,10,Bio-PRF,,1
7,Dec 27 2022 8:55AM,253794,10,ISDIN Corporation,,18
8,Dec 27 2022 8:49AM,253790,10,ISDIN Corporation,,5
9,Dec 27 2022 8:38AM,253789,10,"CLINUVEL, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc.",2,
1,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,40,
2,Dec 27 2022 9:10AM,253802,10,"Methapharm, Inc.",1,
3,Dec 27 2022 9:03AM,253799,10,Bio-PRF,1,
4,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation,61,
5,Dec 27 2022 9:02AM,253797,12,Hush Hush,1,
6,Dec 27 2022 9:01AM,253796,10,Bio-PRF,1,
7,Dec 27 2022 8:55AM,253794,10,ISDIN Corporation,18,
8,Dec 27 2022 8:49AM,253790,10,ISDIN Corporation,5,
9,Dec 27 2022 8:38AM,253789,10,"CLINUVEL, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc.",2,
1,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,40,
2,Dec 27 2022 9:10AM,253802,10,"Methapharm, Inc.",1,
3,Dec 27 2022 9:03AM,253799,10,Bio-PRF,1,
4,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation,61,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc.",2,NaN
1,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,40,NaN
2,Dec 27 2022 9:10AM,253802,10,"Methapharm, Inc.",1,NaN
3,Dec 27 2022 9:03AM,253799,10,Bio-PRF,1,NaN
4,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation,61,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:12AM,253803,10,"Methapharm, Inc.",2,0.0
1,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,40,0.0
2,Dec 27 2022 9:10AM,253802,10,"Methapharm, Inc.",1,0.0
3,Dec 27 2022 9:03AM,253799,10,Bio-PRF,1,0.0
4,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation,61,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2537954,131,0.0
12,253797,1,0.0
19,1015135,5,0.0
20,253784,23,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2537954,131,0.0
1,12,253797,1,0.0
2,19,1015135,5,0.0
3,20,253784,23,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,131,0.0
1,12,1,0.0
2,19,5,0.0
3,20,23,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,131.0
1,12,Released,1.0
2,19,Released,5.0
3,20,Released,23.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20
Status,,,,
Executing,0.0,0.0,0.0,6.0
Released,131.0,1.0,5.0,23.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20
0,Executing,0.0,0.0,0.0,6.0
1,Released,131.0,1.0,5.0,23.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20
0,Executing,0.0,0.0,0.0,6.0
1,Released,131.0,1.0,5.0,23.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()